In [ ]:
import pandas as pd
from math import pi
import numpy as np
np.random.seed(42)
ranges=[
    (-pi,pi),
    (-5*pi/6,5*pi/6),
    (-pi,pi),
    (-5*pi/6,5*pi/6),
    (-pi,pi),
    (-5*pi/6,5*pi/6),
    (-pi,pi),
]
d={}
n=100
for i in range(len(ranges)):
  d["q_%i"%i]=np.random.uniform(ranges[i][0],ranges[i][1],n)
d=pd.DataFrame(d)
print(list(d.index))

In [ ]:
import sympy as sp
pitch=sp.symbols("theta")
roll=sp.symbols("beta")
yaw=sp.symbols("gamma")
m_pitch=sp.Matrix([[1,0,0],[0,sp.cos(pitch),-sp.sin(pitch)],[0,sp.sin(pitch),sp.cos(pitch)]])
m_roll=sp.Matrix([[sp.cos(roll),0,sp.sin(roll)],[0,1,0],[-sp.sin(roll),0,sp.cos(roll)]])
m_yaw=sp.Matrix([[sp.cos(yaw),-sp.sin(yaw),0],[sp.sin(yaw),sp.cos(yaw),0],[0,0,1]])
mm=m_pitch*m_roll*m_yaw
sp.pprint(mm.applyfunc(sp.simplify))

In [ ]:
dd = pd.read_excel("datos7dof.xlsx", sheet_name="Datos",usecols=['x','y','z','pitch','roll','yaw'])
print(dd)

In [ ]:
import numpy as np
import random


class Particle(object):
    def __init__(self, bounds):
        self._x = np.zeros(len(bounds))
        for idx, (lo, hi) in enumerate(bounds):
            self._x[idx] = random.uniform(lo, hi)

        self._best = self._x.copy()
        self._best_value = np.NaN

    def __str__(self):
        return str(self._x)

    @property
    def best(self):
        return self._best

    def set_best(self, x):
        for i, v in enumerate(x):
            self._best[i] = v

    @property
    def best_value(self):
        return self._best_value

    def set_best_value(self, v):
        self._best_value = v

    def __getitem__(self, key):
        return self._x[key]

    def __setitem__(self, key, val):
        self._x[key] = val


class Swarm(object):
    def __init__(self, size, dim, bounds):
        self._particles = [Particle(bounds) for i in range(0, size)]
        self._dim = dim
        self._gbest_value = None
        self._gbest = None

    def size(self):
        return len(self._particles)

    def particles(self):
        return self._particles

    def mean_best(self):
        x = np.zeros(self._dim)
        for p in self._particles:
            x += p.best

        x /= self.size()

        return x

    @property
    def gbest(self):
        return self._gbest

    @property
    def gbest_value(self):
        return self._gbest_value

    def update_gbest(self):
        get_best_value = lambda x: x.best_value
        pg = min(self.particles(), key=get_best_value)
        if (not self._gbest_value) or self._gbest_value > pg.best_value:
            self._gbest = pg.best.copy()
            self._gbest_value = pg.best_value


class QPSO(Swarm):
    def __init__(self, cf, size, dim, bounds, maxIters):
        super(QPSO, self).__init__(size, dim, bounds)
        self._cf = cf
        self._maxIters = maxIters
        self._iters = 0
        self.init_eval()

    def init_eval(self):
        for p in self.particles():
            pc = self._cf(p[:])
            p.set_best_value(pc)

        self.update_gbest()

    def update_best(self):
        for p in self.particles():
            pc = self._cf(p[:])
            if pc < p.best_value:
                p.set_best(p[:])
                p.set_best_value(pc)

        self.update_gbest()

    def kernel_update(self, **kwargs):
        pass

    def update(self, callback=None, interval=None):
        while self._iters <= self._maxIters:
            self.kernel_update()
            self.update_best()
            if callback and (self._iters % interval == 0):
                callback(self)
            if self.gbest_value<1e-3:
              break

            self._iters += 1

    @property
    def iters(self):
        return self._iters

    @property
    def maxIters(self):
        return self._maxIters


class QDPSO(QPSO):
    def __init__(self, cf, size, dim, bounds, maxIters, g):
        super(QDPSO, self).__init__(cf, size, dim, bounds, maxIters)
        self._g = g

    def kernel_update(self, **kwargs):
        for p in self._particles:
            for i in range(0, self._dim):
                u1 = random.uniform(0., 1.)
                u2 = random.uniform(0., 1.)
                u3 = random.uniform(0., 1.)
                rand_sign = 1 if random.random() > 0.5 else -1
                c = (u1 * p.best[i] + u2 * self._gbest[i]) / (u1 + u2)
                L = (1 / self._g) * abs(p[i] - c)
                p[i] = c + rand_sign * L * np.log(1. / u3)

In [ ]:
def sphere(args):
    f = sum([np.power(x, 2.) for x in args])
    return f


def log(s):
    best_value = [p.best_value for p in s.particles()]
    best_value_avg = np.mean(best_value)
    best_value_std = np.std(best_value)
    print("{0: >5}  {1: >9}  {2: >9}  {3: >9}".format("Iters.", "Best", "Best(Mean)", "Best(STD)"))
    print("{0: >5}  {1: >9.3E}  {2: >9.3E}  {3: >9.3E}".format(s.iters, s.gbest_value, best_value_avg, best_value_std))


NParticle = 40
MaxIters = 1000
NDim = 10
bounds = [(-2.56, 5.12) for i in range(0, NDim)]
g = 0.96
s = QDPSO(sphere, NParticle, NDim, bounds, MaxIters, g)
#s.update(callback=log, interval=100)
s.update()
print(s.gbest)
print(s.iters)
print(s.gbest_value)

In [ ]:
!pip install pygad

In [ ]:
import pygad
import numpy as np
function_inputs = [4,-2,3.5,5,-11,-4.7]
desired_output = 44
def fitness_func(ga_instance, solution, solution_idx):
    output = np.sqrt(np.sum(solution**2))
    fitness = 1-np.abs(output)
    return fitness
fitness_function = fitness_func

num_generations = 1000
num_parents_mating = 4

sol_per_pop = 8
num_genes = len(function_inputs)

init_range_low = -np.pi
init_range_high = np.pi

gene_space = [{'low': 1, 'high': 5}, {'low': -1, 'high': 0}, {'low': 0, 'high': 4}, {'low': 0.5, 'high': 2}, {'low': 1, 'high': 5}, {'low': 0, 'high': 1}]

parent_selection_type = "sss"
keep_parents = 1

crossover_type = "single_point"

mutation_type = "random"
mutation_percent_genes = 10

stop_criteria=["reach_0.999"]

ga_instance = pygad.GA(num_generations=num_generations,
                       num_parents_mating=num_parents_mating,
                       fitness_func=fitness_function,
                       sol_per_pop=sol_per_pop,
                       num_genes=num_genes,
                       init_range_low=init_range_low,
                       init_range_high=init_range_high,
                       parent_selection_type=parent_selection_type,
                       keep_parents=keep_parents,
                       crossover_type=crossover_type,
                       mutation_type=mutation_type,
                       mutation_percent_genes=mutation_percent_genes,
                       gene_space=gene_space,
                       stop_criteria=stop_criteria
                       )

ga_instance.run()
solution, solution_fitness, solution_idx = ga_instance.best_solution()
print("Parameters of the best solution : {solution}".format(solution=solution))
print("Fitness value of the best solution = {solution_fitness}".format(solution_fitness=solution_fitness))
print(solution_idx)
prediction = np.sum(np.array(function_inputs)*solution)
print("Predicted output based on the best solution : {prediction}".format(prediction=prediction))

In [ ]:
!pip install pyswarms

In [ ]:
import numpy as np

# Import sphere function as objective function
from pyswarms.utils.functions.single_obj import sphere as f

# Import backend modules
import pyswarms.backend as P
from pyswarms.backend.topology import Star

my_topology = Star() # The Topology Class
bounds=[(-1,-1,0),(0.0001,2,1)]
my_options = {'c1': 0.6, 'c2': 0.3, 'w': 0.4} # arbitrarily set
my_swarm = P.create_swarm(n_particles=50, dimensions=3, options=my_options,bounds=bounds) # The Swarm Class

print('The following are the attributes of our swarm: {}'.format(my_swarm.__dict__.keys()))

iterations = 1000 # Set 100 iterations
for i in range(iterations):
    # Part 1: Update personal best
    my_swarm.current_cost = f(my_swarm.position) # Compute current cost
    my_swarm.pbest_cost = f(my_swarm.pbest_pos)  # Compute personal best pos
    my_swarm.pbest_pos, my_swarm.pbest_cost = P.compute_pbest(my_swarm) # Update and store

    # Part 2: Update global best
    # Note that gbest computation is dependent on your topology
    if np.min(my_swarm.pbest_cost) < my_swarm.best_cost:
        my_swarm.best_pos, my_swarm.best_cost = my_topology.compute_gbest(my_swarm)

    if my_swarm.best_cost<1e-3:
      break

    # Let's print our output
    if i%20==0:
        print('Iteration: {} | my_swarm.best_cost: {:.4f}'.format(i+1, my_swarm.best_cost))

    # Part 3: Update position and velocity matrices
    # Note that position and velocity updates are dependent on your topology
    my_swarm.velocity = my_topology.compute_velocity(my_swarm)
    my_swarm.position = my_topology.compute_position(my_swarm)

print('The best cost found by our swarm is: {:.4f}'.format(my_swarm.best_cost))
print('The best position found by our swarm is: {}'.format(my_swarm.best_pos))